In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import re
import nltk
import pandas as pd
import seaborn as sns
import numpy as np
import string
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from sklearn.linear_model import SGDClassifier,LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier

In [3]:
tweets= pd.read_csv("/content/drive/MyDrive/input/twitter_training.csv",header=None)
tweets_test = pd.read_csv('/content/drive/MyDrive/input/twitter_validation.csv',header=None)

In [4]:
tweets=tweets[[2,3]]
tweets.columns = ["sentiment","text"]
tweets.dropna(inplace=True)
tweets.drop_duplicates(inplace=True)


<ipython-input-4-4a462bd5e6a7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets.dropna(inplace=True)
<ipython-input-4-4a462bd5e6a7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets.drop_duplicates(inplace=True)


In [5]:
tweets.sentiment.value_counts(normalize=True)

,proportion
sentiment,
Negative,0.304390
Positive,0.274305
Neutral,0.245238
Irrelevant,0.176067


In [6]:
tweets.loc[tweets["sentiment"]=='Irrelevant','sentiment']='Neutral'

<ipython-input-6-08c81265a69b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets.loc[tweets["sentiment"]=='Irrelevant','sentiment']='Neutral'


In [7]:
tweets.sentiment.value_counts(normalize=True)


,proportion
sentiment,
Neutral,0.421305
Negative,0.304390
Positive,0.274305


In [8]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [9]:
#pos tag,lemmatization,removing stopwords

from textblob import TextBlob
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk import pos_tag
from nltk.corpus import wordnet
stopWords = set(stopwords.words('english'))-{'no','not','neither'}
def nltk_to_wordnet(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def pos_tagging_and_lemmatization(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)
    lemmatized_words = []
    for word, tag in tagged:
        wn_tag = nltk_to_wordnet(tag)
        if wn_tag is None:
            lemmatized_word = lemmatizer.lemmatize(word)
        else:
            lemmatized_word = lemmatizer.lemmatize(word, pos=wn_tag)
        lemmatized_words.append(lemmatized_word)

    return lemmatized_words

def text_preprocessing(text):
    clean_text=[]
    for sentence in tqdm(text):
        sentence = re.sub(r'(https?://\S+|www\.\S+|\b\w+\.\w+\b)','',sentence)
        sentence = re.sub(r'@\s*\S+',' ',sentence)
        sentence= re.sub(r'https:\S+',' ',sentence)
        sentence=re.sub(r'\s+',' ',sentence)
        sentence = re.sub(r"[^a-zA-Z]+",' ',sentence).lower().strip()
        lemma_words = pos_tagging_and_lemmatization(sentence)
        sentence = ' '.join(lemma_words)
        clean_text.append(sentence.strip())
    return clean_text



clean_text=text_preprocessing(tweets.text.values)


tfidf = TfidfVectorizer(ngram_range=(1,2),min_df=10,max_features=300000)
tf_count=tfidf.fit_transform(clean_text)

100%|██████████| 69769/69769 [01:39<00:00, 701.05it/s]


In [10]:
from sklearn import metrics
from sklearn.metrics import classification_report
y= tweets.sentiment.astype('category').cat.codes
X_train,X_test,y_train,y_test = train_test_split(tf_count,y,test_size=0.2,random_state=42)
xg = LogisticRegression(solver='saga',max_iter=5000,multi_class='multinomial')
xg.fit(X_train,y_train)
y_train_preds = xg.predict(X_train)
y_test_preds = xg.predict(X_test)
print(classification_report(y_train,y_train_preds))
print(classification_report(y_test,y_test_preds))

              precision    recall  f1-score   support

           0       0.90      0.89      0.90     16975
           1       0.87      0.91      0.89     23425
           2       0.90      0.85      0.87     15415

    accuracy                           0.89     55815
   macro avg       0.89      0.88      0.89     55815
weighted avg       0.89      0.89      0.89     55815

              precision    recall  f1-score   support

           0       0.82      0.81      0.82      4262
           1       0.81      0.85      0.83      5969
           2       0.82      0.76      0.79      3723

    accuracy                           0.82     13954
   macro avg       0.82      0.81      0.81     13954
weighted avg       0.82      0.82      0.82     13954



In [11]:
X_train.shape,X_test.shape

((55815, 22110), (13954, 22110))

In [12]:
xg = RandomForestClassifier(n_estimators=175,criterion='entropy',oob_score=True,n_jobs=30)
xg.fit(X_train,y_train)
y_train_preds = xg.predict(X_train)
y_test_preds = xg.predict(X_test)
print(classification_report(y_train,y_train_preds))
print(classification_report(y_test,y_test_preds))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     16975
           1       0.99      0.99      0.99     23425
           2       0.99      0.99      0.99     15415

    accuracy                           0.99     55815
   macro avg       0.99      0.99      0.99     55815
weighted avg       0.99      0.99      0.99     55815

              precision    recall  f1-score   support

           0       0.94      0.90      0.92      4262
           1       0.89      0.96      0.92      5969
           2       0.94      0.87      0.90      3723

    accuracy                           0.92     13954
   macro avg       0.92      0.91      0.92     13954
weighted avg       0.92      0.92      0.92     13954

